[2024-10-04 Fiddler](https://thefiddler.substack.com/p/how-many-dice-can-you-roll-the-same)
====================

It's not specified anywhere, so I'll assume that the dice are 6-sided.

The question seems ambiguous.  I can see three interpretations.

The obviously wrong interpretation has an easy answer.  The total number of dice set aside per player
per game on average will be 3, or, for the extra credit, 10 or $N$.

The second interpretation is the number of dice set aside from the first roll on average.
This is one roll per player per game.

The third interpretation is the average number of dice set aside per roll from each roll in the game,
including the first roll.

The wording seems to seems to imply the second interpretation.  Also, the first and third interpretations
can only be calculated for the winning player if the losing players stop rolling once they lose.  (For the
second interpretation, a very slow losing player might lose before setting aside the dice from the first
roll, and thus not contribute to the average.)

Simulation
----------
To get a sense of what the answers should be, run some [simulations](20241004.go), calculating both
the average number of dice set aside from the first roll of a game and the average number of dice
set aside per roll from every roll.

    $ go run 20241004.go
    3: 1.472164 0.359562
    10: 3.445436 0.651536
    25: 6.899469 1.208623
    50: 12.143944 2.026984
    100: 21.998333 3.496237

Fiddler
-------

The probability of rolling 3 of the same number is $1/36$.  The probability of rolling 3 different
numbers is $20/36$, so the probability of rolling 2 of one number and 1 of a different number is
$15/36$.  That means the average number of dice set aside from the first roll is

In [1]:
x = 3*1/36 + 2*15/36 + 1*20/36
(x,numerical_approx(x))

(53/36, 1.47222222222222)

which agrees with the result of the simulations.

When there is 1 die left, the average number of rolls is $6$.

When there are 2 dice left, the probability of setting aside 2 dice in a roll is $1/36$.  The
probability of setting aside 1 die is $10/36$.  The probability of setting aside none is $25/36$,
so the average number of rolls is

In [2]:
x = var('x')
solve(x == 1 + 0*1/36 + 6*10/36 + x*25/36, x)

[x == (96/11)]

The overall average number of rolls is then

In [3]:
1*1/36 + (1 + 6)*15/36 + (1 + 96/11)*20/36

551/66

So the overall average number of dice set aside per roll is

In [4]:
x = 3/(551/66)
(x,numerical_approx(x))

(198/551, 0.359346642468240)

which agrees with the result of the simulations.

Extra credit
------------

Let $P(N,n,k)$ be the probability that the result rolling $N$ dice has $k$ groups of
$n$ dice with the same number, and all groups of dice with any other numbers are smaller
than $n$.

The average number of dice set aside from the first roll is
$\sum_{n=1}^{N} \sum_{k=1}^{6} nP(N,n,k)$.

Let $R(n)$ be the average number of rolls to set aside $n$ dice after the target is set.

The average number of dice set aside per roll is
$N/\sum_{n=1}^{N} \sum_{k=1}^{6} (1+R(N-n))P(N,n,k)$.

#### Consider $N=10$
To get a better sense of how to approach the general case, consider $N=10$.

Define `nbinomial` and `dbinomial` as `binomial` to distinguish between considering
combinations of numbers and combinations of dice.

In [5]:
nbinomial = binomial
dbinomial = binomial

The probabilities of setting aside 6, 7, 8, 9, and 10 from the first roll are

In [6]:
p10 = nbinomial(6,1)*(1/6)^10*dbinomial(10,10)
p9 = nbinomial(6,1)*(1/6)^9*dbinomial(10,9)*(5/6)
p8 = nbinomial(6,1)*(1/6)^8*dbinomial(10,8)*(5/6)^2
p7 = nbinomial(6,1)*(1/6)^7*dbinomial(10,7)*(5/6)^3
p6 = nbinomial(6,1)*(1/6)^6*dbinomial(10,6)*(5/6)^4
[(p,numerical_approx(p)) for p in [p10,p9,p8,p7,p6]]

[(1/10077696, 9.92290301275212e-8),
 (25/5038848, 4.96145150637606e-6),
 (125/1119744, 0.000111632658893461),
 (625/419904, 0.00148843545191282),
 (21875/1679616, 0.0130238102042372)]

The probability of rolling 5 of one number and 5 of another number is

In [7]:
p5_2 = nbinomial(6,2)*(1/6)^10*dbinomial(10,5)
(p5_2,numerical_approx(p5_2))

(35/559872, 0.0000625142889803384)

The probability of rolling 5 of one number and less than 5 of every other number is

In [8]:
p5_1 = nbinomial(6,1)*(1/6)^5*dbinomial(10,5)*((5/6)^5 - (1/6)^5*nbinomial(5,1))
(p5_1, numerical_approx(p5_1))

(455/5832, 0.0780178326474623)

In [9]:
p5 = p5_1+p5_2
(p5, numerical_approx(p5))

(43715/559872, 0.0780803469364426)

The probability of rolling 4 of one number and 4 of another number is

In [10]:
p4_2 = nbinomial(6,2)*(1/6)^8*dbinomial(8,4)*(4/6)^2*dbinomial(10,2)
(p4_2, numerical_approx(p4_2))

(875/69984, 0.0125028577960677)

The probability of rolling 4 of one number and less than 4 of any other number is

In [11]:
p4_1 = nbinomial(6,1)*(1/6)^4*dbinomial(10,4)*((5/6)^6
            - (1/6)^6*nbinomial(5,1)
            - (1/6)^5*nbinomial(5,1)*(4/6)*dbinomial(6,1)
            - (1/6)^4*nbinomial(5,1)*(4/6)^2*dbinomial(6,2))
(p4_1, numerical_approx(p4_1))

(125125/419904, 0.297984777472946)

In [12]:
p4 = p4_2 + p4_1
(p4, numerical_approx(p4))

(130375/419904, 0.310487635269014)

The probability of rolling 3 of three numbers and 1 of a fourth number is

In [13]:
p3_3_1 = nbinomial(6,3)*(1/6)^9*dbinomial(6,3)*dbinomial(9,3)*(3/6)*dbinomial(10,1)
(p3_3_1,numerical_approx(p3_3_1))

(875/52488, 0.0166704770614236)

The probability of rolling 3 of two numbers and less than 3 of any other number is

In [14]:
p3_2 = nbinomial(6,2)*(1/6)^6*dbinomial(6,3)*dbinomial(10,6)*((4/6)^4
            - (1/6)^4*nbinomial(4,1)
            - (1/6)^3*nbinomial(4,1)*(3/6)*dbinomial(4,1))
(p3_2,numerical_approx(p3_2))

(14875/69984, 0.212548582533150)

The probability of rolling 3 of one number and less than 3 of any other number is

In [15]:
p3_1 = nbinomial(6,1)*(1/6)^3*dbinomial(10,3)*((5/6)^7
            - (1/6)^7*nbinomial(5,1)
            - (1/6)^6*nbinomial(5,1)*(4/6)*dbinomial(7,1)
            - (1/6)^5*nbinomial(5,1)*(4/6)^2*dbinomial(7,2)
            - (1/6)^4*nbinomial(5,1)*(4/6)^3*dbinomial(7,3)
            - (1/6)^6*nbinomial(5,2)*dbinomial(6,3)*(3/6)*dbinomial(7,1)
            - (1/6)^3*nbinomial(5,1)*((4/6)^4
                        - (1/6)^4*nbinomial(4,1)
                        - (1/6)^3*nbinomial(4,1)*(3/6)*dbinomial(4,1))*dbinomial(7,4))
(p3_1,numerical_approx(p3_1))

(875/2916, 0.300068587105624)

In [16]:
p3 = p3_3_1 + p3_2 + p3_1
(p3,numerical_approx(p3))

(111125/209952, 0.529287646700198)

The probability of rolling 2 of five numbers is

In [17]:
p2_5 = nbinomial(6,5)*(1/6)^10*dbinomial(4,2)*dbinomial(6,2)*dbinomial(8,2)*dbinomial(10,2)
(p2_5,numerical_approx(p2_5))

(175/15552, 0.0112525720164609)

The probability of rolling 2 of four numbers and 1 of each of the other two numbers is

In [18]:
p2_4_2 = nbinomial(6,4)*(1/6)^8*dbinomial(4,2)*dbinomial(6,2)*dbinomial(8,2)*(2/6)*(1/6)*dbinomial(10,2)
(p2_4_2,numerical_approx(p2_4_2))

(875/15552, 0.0562628600823045)

In [19]:
p2 = p2_5 + p2_4_2
(p2,numerical_approx(p2))

(175/2592, 0.0675154320987654)

In [20]:
p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10

1

The average number of dice set aside from the first roll of 10 dice is

In [21]:
x = 2*p2 + 3*p3 + 4*p4 + 5*p5 + 6*p6 + 7*p7 + 8*p8 + 9*p9 + 10*p10
(x,numerical_approx(x))

(17357555/5038848, 3.44474669607021)

which agrees with the results of the simulations.

#### Calculating $R$

In [22]:
from functools import cache

@cache
def R_recursive(n):
    if n == 0:
        return 0
    denominator = 1 - (5/6)^n*binomial(n,0)
    numerator = 1 + sum([R_recursive(n-k)*((1/6)^k*(5/6)^(n-k)*binomial(n,k)) for k in [1..n]])
    return numerator/denominator

In [23]:
k = var('k')
R(n) = sum(binomial(n,k)*(-1)^(k+1)/(1-(5/6)^k),k,1,n)

In [24]:
[(R_recursive(k) - R(k).simplify(),R(k).simplify()) for k in [1..10]]

[(0, 6),
 (0, 96/11),
 (0, 10566/1001),
 (0, 728256/61061),
 (0, 3698650986/283994711),
 (0, 9438928992/677218157),
 (0, 3736509841479198/253815850752893),
 (0, 7512121341839614848/487580249296307453),
 (0, 443684776375651380574086/27701960514724125452641),
 (0, 5002237679837182013478909216/301979071571007691559239541)]

And the average number of dice set aside per roll for $N=10$ is

In [25]:
x = (10/((1+R(8))*p2 + (1+R(7))*p3 + (1+R(6))*p4 + (1+R(5))*p5 + (1+R(4))*p6
        + (1+R(3))*p7 + (1+R(2))*p8 + (1+R(1))*p9 + (1+R(0))*p10)).simplify()
(x,numerical_approx(x))

(447861962188630247878620/687400143797944474874777, 0.651530213121799)

which agrees with the results of the simulations.

### General case

... I may come back to this some day ...

In [26]:
j,k,m = var('j,k,m')
P(N,n,k) = max_symbolic(0,2-ceil(k*n/N))*binomial(6,k)*(1/6)^(k*n)*product(binomial(N-(j-1)*n,n),j,1,k)*(
    (1-k/6)^(N-k*n) - sum(0,j,n,N-k*n)) # TODO: put the probability of group of size j (out of N-k*n) in the sum
    # TODO: support N=multiple of 6: fix division by zero for k=6 (P(N,n,k) = 0 unless n = N/6)
show(P)

(N, n, k) |--> (1/6)^(k*n)*(-1/6*k + 1)^(-k*n + N)*binomial(6, k)*max(0, -ceil(k*n/N) + 2)*product(binomial(-(j - 1)*n + N, n), j, 1, k)

In [27]:
def check(N,n,k,p):
    pp = P(N,n,k).simplify()
    if pp != p:
        print(f"P({N},{n},{k})={pp}, should be {p}")

In [28]:
check(3,1,1,0)
check(3,1,2,0)
check(3,1,3,20/36)
check(3,1,4,0)
check(3,1,5,0)
check(3,2,1,15/36)
check(3,2,2,0)
check(3,2,3,0)
check(3,2,4,0)
check(3,2,5,0)
check(3,3,1,1/36)
check(3,3,2,0)
check(3,3,3,0)
check(3,3,4,0)
check(3,3,5,0)

P(3,1,1)=25/12, should be 0
P(3,1,2)=5/3, should be 0


In [29]:
check(10,1,1,0)
check(10,1,2,0)
check(10,1,3,0)
check(10,1,4,0)
check(10,1,5,0)
check(10,2,1,0)
check(10,2,2,0)
check(10,2,3,0)
check(10,2,4,p2_4_2)
check(10,2,5,p2_5)
check(10,3,1,p3_1)
check(10,3,2,p3_2)
check(10,3,3,p3_3_1)
check(10,3,4,0)
check(10,3,5,0)
check(10,4,1,p4_1)
check(10,4,2,p4_2)
check(10,4,3,0)
check(10,4,4,0)
check(10,4,5,0)
check(10,5,1,p5_1)
check(10,5,2,p5_2)
check(10,5,3,0)
check(10,5,4,0)
check(10,5,5,0)
check(10,6,1,p6)
check(10,6,2,0)
check(10,6,3,0)
check(10,6,4,0)
check(10,6,5,0)
check(10,7,1,p7)
check(10,7,2,0)
check(10,7,3,0)
check(10,7,4,0)
check(10,7,5,0)
check(10,8,1,p8)
check(10,8,2,0)
check(10,8,3,0)
check(10,8,4,0)
check(10,8,5,0)
check(10,9,1,p9)
check(10,9,2,0)
check(10,9,3,0)
check(10,9,4,0)
check(10,9,5,0)
check(10,10,1,p10)
check(10,10,2,0)
check(10,10,3,0)
check(10,10,4,0)
check(10,10,5,0)

P(10,1,1)=9765625/5038848, should be 0
P(10,1,2)=3200/2187, should be 0
P(10,1,3)=25/48, should be 0
P(10,1,4)=175/2187, should be 0
P(10,1,5)=35/11664, should be 0
P(10,2,1)=1953125/1119744, should be 0
P(10,2,2)=2800/2187, should be 0
P(10,2,3)=875/1728, should be 0
P(10,2,4)=875/7776, should be 875/15552
P(10,3,1)=390625/419904, should be 875/2916
P(10,3,2)=1750/6561, should be 14875/69984
P(10,4,1)=546875/1679616, should be 125125/419904
P(10,5,1)=21875/279936, should be 455/5832


In [30]:
# TODO: support N = multiple of 6
from_first_roll(N) = sum(sum(n*P(N,n,k),k,1,5),n,1,N)
from_all_rolls(N) = N/sum((1+R(N-n))*sum(P(N,n,k),k,1,5),n,1,N)

In [31]:
def check_calculations(N, first, all):
    print(f"{sum(sum(P(N,n,k,),k,1,5),n,1,N).simplify()} should be 1") # TODO: k should go from 1 to 6
    x = from_first_roll(N).simplify()
    print(f"{(x,numerical_approx(x))} should be {first}")
    x = from_all_rolls(N).simplify()
    print(f"{(x,numerical_approx(x))} should be {all}")

In [32]:
check_calculations(3, 53/36, 198/551)
check_calculations(10, 17357555/5038848, 447861962188630247878620/687400143797944474874777)
check_calculations(25, 6.899469, 1.208623)
check_calculations(50, 12.143944, 2.026984)
check_calculations(100, 21.998333, 3.496237)

19/4 should be 1
(47/9, 5.22222222222222) should be 53/36
(396/5917, 0.0669258069967889) should be 198/551
93767021/10077696 should be 1
(348125/20736, 16.7884355709877) should be 17357555/5038848
(4386987576264751231361946209280/67353074275959464046596099678413, 0.0651341846444953) should be 447861962188630247878620/687400143797944474874777
4476495500696238319/526486815369068544 should be 1
(1740035335430790475/49358138940850176, 35.2532606125206) should be 6.89946900000000
(4460633965968188493970231707102423981633052718274246556429044631540480825791204063997931887953703426381011015993825109298195522651422370498588452465348391731200/32525776167093049094629290818788359489421620971276936790379031803785016489767800746847423269043709964764268118445664574985515696079610463188862464979071526356483, 0.137141507186571) should be 1.20862300000000
1037918955720128820066822827491409150671/134713546244127343440523266742756048896 should be 1
(717609942951088286812065452111231048225/112261288536772

Making the rounds
-----------------

### Arithmetic

In [33]:
%display latex
a1,a2,a3 = var('a1,a2,a3')
solve(a2 == (a1+a3)/2, a3)

[a3 == -a1 + 2*a2]

In [34]:
a(i) = a1 + (a2-a1)*i

In [35]:
(a(i+1) == (a(i)+a(i+2))/2).simplify_full()

-(a1 - a2)*i + a2 == -(a1 - a2)*i + a2

In [36]:
a

i |--> -(a1 - a2)*i + a1

### Geometric

In [37]:
g1,g2,g3 = var('g1,g2,g3')
assume(g2 > 0)
solve(g2 == sqrt(g1*g3), g3)

[g3 == g2^2/g1]

In [38]:
g(i) = g1*(g2/g1)^i

In [39]:
assume(g1 > 0)
(g(i+1) == sqrt(g(i)*g(i+2))).simplify_full()

g2*(g2/g1)^i == g2*(g2/g1)^i

In [40]:
g

i |--> g1*(g2/g1)^i

### Quadratic

In [41]:
q1,q2,q3 = var('q1,q2,q3')
assume(q2 > 0)
solve(q2 == sqrt((q1^2+q3^2)/2), q3)

[q3 == -sqrt(-q1^2 + 2*q2^2), q3 == sqrt(-q1^2 + 2*q2^2)]

In [42]:
sqrt(-q2^2 + 2*(-q1^2+2*q2^2))

sqrt(-2*q1^2 + 3*q2^2)

In [43]:
sqrt(-(-q1^2+2*q2^2)+2*(-2*q1^2+3*q2^2))

sqrt(-3*q1^2 + 4*q2^2)

In [44]:
q(i) = sqrt(q2^2-i*(q1^2 - q2^2))

In [45]:
(q(i+1) == sqrt((q(i)^2+q(i+2)^2)/2)).simplify_full()

sqrt(-(i + 1)*q1^2 + (i + 2)*q2^2) == sqrt(-(i + 1)*q1^2 + (i + 2)*q2^2)

In [46]:
q

i |--> sqrt(-(q1^2 - q2^2)*i + q2^2)

We could also arbitrarily pick the negative root for any of the terms and it would still satisfy
the condition.

### Harmonic

In [47]:
h1,h2,h3 = var('h1,h2,h3')
solve(h2 == 2/(1/h1+1/h3),h3)

[h3 == h1*h2/(2*h1 - h2)]

In [48]:
((h2*h3)/(2*h2-h3)).substitute(h3 == h1*h2/(2*h1-h2)).simplify_full()

h1*h2/(3*h1 - 2*h2)

In [49]:
x1,x2 = var('x1,x2')
(x1*x2/(2*x1-x2)).substitute(x1 == h2, x2 == h1*h2/(2*h1-h2)).simplify_full()

h1*h2/(3*h1 - 2*h2)

In [50]:
(x1*x2/(2*x1-x2)).substitute(x1 == h1*h2/(2*h1-h2), x2 == h1*h2/(3*h1-2*h2)).simplify_full()

h1*h2/(4*h1 - 3*h2)

In [51]:
h(i) = h1*h2/(h1 + i*(h1-h2))

In [52]:
(h(i+1) == 2/(1/h(i)+1/h(i+2))).simplify_full()

h1*h2/((h1 - h2)*i + 2*h1 - h2) == h1*h2/((h1 - h2)*i + 2*h1 - h2)

This blows up when $h_1 + i(h_1-h_2) = 0$,

In [53]:
solve(h1 + i*(h1-h2) == 0, i)

[i == -h1/(h1 - h2)]

So if $h_1/(h_1-h_2)$ is an integer, infinity will be part of the sequence and still satisfy the condition.

In [54]:
h

i |--> h1*h2/((h1 - h2)*i + h1)